# DMA Crossover + Two-Green Entry / Two-Red Exit — v2 (No Plots by Default)
Adds **optional filters** to reduce false/negative trades and disables plotting by default.
Pure Python (`pandas`, `yfinance`, `matplotlib` only if you turn plots back on).

### Optional Filters You Can Toggle
- Price above 200DMA (avoid weak regimes)
- 20DMA slope positive (avoid flat/whipsaw)
- RSI filter (e.g., RSI>50)
- ADX filter (e.g., ADX>18–25 for trend strength)
- Volume surge vs. 20-day average
- ATR band: skip too-low or too-high volatility regimes
- Protective exit: close below 20DMA exits at next open (prevents waiting for two red candles)


## Parameters (edit me)

In [19]:

# ====================== USER PARAMETERS ======================
TICKERS = ['SBICARD.NS', 'BDL.NS', 'INDHOTEL.NS', 'BSE.NS', 'NYKAA.NS', 'BAJFINANCE.NS', 'PAYTM.NS', 'SOLARINDS.NS', 'CHOLAFIN.NS', 'UNITDSPR.NS', 'DIVISLAB.NS', 'MUTHOOTFIN.NS', 'BHARTIARTL.NS', 'ICICIBANK.NS', 'MAZDOCK.NS', 'SHREECEM.NS', 'DIXON.NS', 'PERSISTENT.NS', 'SRF.NS', 'TVSMOTOR.NS', 'SBILIFE.NS', 'MAXHEALTH.NS', 'MFSL.NS', 'COFORGE.NS', 'HDFCLIFE.NS', 'INDIGO.NS', 'KOTAKBANK.NS', 'HDFCBANK.NS', 'BEL.NS', 'BAJAJFINSV.NS']


START_DATE = "2025-01-01"
END_DATE   = None

FAST_DMA = 10
SLOW_DMA = 20
REGIME_DMA = 200  # for regime filter

# Entry timing relative to cross
REQUIRE_CROSS_EXACTLY_T_MINUS_2 = True
CROSS_WINDOW_AFTER = 3  # used only if above flag is False

TZ = "Asia/Kolkata"

# Backtest settings
INITIAL_CAPITAL = 50000.0
RISK_PER_TRADE = 1.0
SLIPPAGE_BPS = 5
FEES_BPS = 2

# -------- Optional Filters (set True/False) --------
USE_FILTER_PRICE_ABOVE_200DMA = False
USE_FILTER_SLOW_SLOPE_POSITIVE = False          # slope of 20DMA positive
USE_FILTER_RSI = False; RSI_LEN = 14; RSI_MIN = 50
USE_FILTER_ADX = False; ADX_LEN = 14; ADX_MIN = 18
USE_FILTER_VOLUME_SURGE = False; VOL_LOOKBACK = 20; VOL_FACTOR = 1.2
USE_FILTER_ATR_BAND = False; ATR_LEN = 14; ATR_MIN_PCT = 0.5; ATR_MAX_PCT = 4.0
    # Require ATR% (ATR/Close*100) to be within [ATR_MIN_PCT, ATR_MAX_PCT]


# Protective exit (optional)
PROTECTIVE_EXIT_ON_CLOSE_BELOW_20DMA = False

# Plot settings (disabled by default)
PLOT_LAST_N = 0        # 0 means no plotting
SAVE_PLOTS = False
PLOTS_DIR = "plots_dma_two_green_v2"
# ============================================================


## Install & Imports

In [20]:

# !pip install yfinance pandas numpy matplotlib

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import os

if SAVE_PLOTS and not os.path.isdir(PLOTS_DIR):
    os.makedirs(PLOTS_DIR, exist_ok=True)


## Helpers: Indicators (RSI/ATR/ADX), Data, Signals, Backtest

In [21]:

def _to_tz_local_naive(df, tz):
    if df.index.tz is None:
        df.index = df.index.tz_localize('UTC').tz_convert(tz).tz_localize(None)
    else:
        df.index = df.index.tz_convert(tz).tz_localize(None)
    return df

def fetch_1d(ticker, start, end, tz):
    df = yf.download(ticker, start=start, end=end, interval='1d', auto_adjust=False, progress=False, multi_level_index=False)
    if df.empty: return df
    df = df.rename(columns=str.title)
    df = _to_tz_local_naive(df, tz).dropna(subset=['Open','High','Low','Close','Volume'])
    return df

def SMA(s, n):
    return s.rolling(n, min_periods=n).mean()

def RSI(close, n=14):
    delta = close.diff()
    gain = delta.clip(lower=0.0)
    loss = -delta.clip(upper=0.0)
    # Wilder's smoothing
    avg_gain = gain.ewm(alpha=1/n, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/n, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(50)  # neutral when undefined

def ATR(high, low, close, n=14):
    prev_close = close.shift(1)
    tr = pd.concat([
        (high - low),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)
    return tr.ewm(alpha=1/n, adjust=False).mean()

def ADX(high, low, close, n=14):
    # Wilder's ADX
    up_move = high.diff()
    down_move = -low.diff()
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    tr = pd.concat([
        (high - low),
        (high - close.shift(1)).abs(),
        (low - close.shift(1)).abs()
    ], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1/n, adjust=False).mean()
    plus_di = 100 * pd.Series(plus_dm, index=close.index).ewm(alpha=1/n, adjust=False).mean() / atr
    minus_di = 100 * pd.Series(minus_dm, index=close.index).ewm(alpha=1/n, adjust=False).mean() / atr
    dx = ( (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0,np.nan) ) * 100
    adx = dx.ewm(alpha=1/n, adjust=False).mean()
    return adx

def add_indicators(df, fast=10, slow=20, regime=200, rsi_len=14, adx_len=14, atr_len=14):
    o = df.copy()
    o[f'DMA_{fast}'] = SMA(o['Close'], fast)
    o[f'DMA_{slow}'] = SMA(o['Close'], slow)
    o[f'DMA_{regime}'] = SMA(o['Close'], regime)
    o['Green'] = o['Close'] > o['Open']
    o['Red']   = o['Close'] < o['Open']
    o['RSI'] = RSI(o['Close'], rsi_len)
    o['ATR'] = ATR(o['High'], o['Low'], o['Close'], atr_len)
    o['ATR_PCT'] = o['ATR'] / o['Close'] * 100
    o['ADX'] = ADX(o['High'], o['Low'], o['Close'], adx_len)
    # Slope of slow DMA (first difference)
    o['SLOW_SLOPE'] = o[f'DMA_{slow}'].diff()
    o['VOL_MA'] = SMA(o['Volume'], 20)
    return o

def mark_crosses(out, fast=10, slow=20):
    f, s = f'DMA_{fast}', f'DMA_{slow}'
    out['CrossUp'] = (out[f].shift(1) <= out[s].shift(1)) & (out[f] > out[s])
    out['CrossDn'] = (out[f].shift(1) >= out[s].shift(1)) & (out[f] < out[s])
    return out

def entry_signal(o, fast=10, slow=20,
                 exact_t_minus_2=True, cross_window_after=3,
                 regime_ok=True, slope_ok=True, rsi_ok=True, rsi_min=50,
                 adx_ok=True, adx_min=18, vol_surge=False, vol_lb=20, vol_factor=1.2,
                 atr_band=True, atr_min_pct=0.5, atr_max_pct=4.0):
    green_seq = (o['Green'] & o['Green'].shift(1))  # two greens ending at t
    higher_close = o['Close'] > o['Close'].shift(1)
    base = (green_seq & higher_close)

    # Cross timing
    if exact_t_minus_2:
        cross_cond = o['CrossUp'].shift(2).fillna(False)
    else:
        c = False
        for k in range(2, 2 + cross_window_after):
            c = c | o['CrossUp'].shift(k).fillna(False)
        cross_cond = c

    cond = base & cross_cond

    # Optional filters
    if regime_ok:
        cond &= o['Close'] > o[f'DMA_{200}']
    if slope_ok:
        cond &= o['SLOW_SLOPE'] > 0
    if rsi_ok:
        cond &= o['RSI'] >= rsi_min
    if adx_ok:
        cond &= o['ADX'] >= adx_min
    if vol_surge:
        cond &= o['Volume'] >= (o['VOL_MA'] * vol_factor)
    if atr_band:
        cond &= (o['ATR_PCT'] >= atr_min_pct) & (o['ATR_PCT'] <= atr_max_pct)

    return cond.fillna(False)

def exit_signal(o, slow=20, protective_on_close_below_slow=True):
    red_seq = (o['Red'] & o['Red'].shift(1))
    below_slow = o['Close'] < o[f'DMA_{slow}']
    base_exit = (red_seq & below_slow)
    if protective_on_close_below_slow:
        base_exit = base_exit | below_slow  # any close below 20DMA triggers exit next open
    return base_exit.fillna(False)

def backtest_long_only(df, enter_sig, exit_sig, initial_capital=100000.0, risk_frac=1.0,
                       slippage_bps=5, fees_bps=2):
    ohlc = df[['Open','High','Low','Close','Volume']].copy()
    open_next = ohlc['Open'].shift(-1)
    slippage = slippage_bps / 10000.0
    fees = fees_bps / 10000.0

    in_pos=False; qty=0; entry_price=np.nan; cash=initial_capital
    equity_curve=[]; trades=[]

    for i, (ts,row) in enumerate(ohlc.iterrows()):
        equity = cash + qty * row['Close']
        equity_curve.append((ts,equity))
        if i == len(ohlc)-1: continue

        if (not in_pos) and enter_sig.loc[ts]:
            px = open_next.loc[ts]
            if pd.isna(px) or px<=0: continue
            fill = px * (1+slippage)
            fee_amt = fill * fees
            alloc = equity * risk_frac
            qty = int(alloc // fill)
            if qty<=0: continue
            cash -= qty*fill + fee_amt
            entry_price = fill; in_pos=True
            trades.append({'Time': ohlc.index[i+1], 'Ticker': df.attrs.get('Ticker','N/A'),
                           'Action':'BUY','Qty':qty,'Price':fill,'Fees':fee_amt})
        elif in_pos and exit_sig.loc[ts]:
            px = open_next.loc[ts]
            if pd.isna(px) or px<=0: continue
            fill = px * (1-slippage)
            notional = qty*fill
            fee_amt = notional * fees
            cash += notional - fee_amt
            trades.append({'Time': ohlc.index[i+1], 'Ticker': df.attrs.get('Ticker','N/A'),
                           'Action':'SELL','Qty':qty,'Price':fill,'Fees':fee_amt,
                           'PnL': (fill-entry_price)*qty - fee_amt})
            qty=0; entry_price=np.nan; in_pos=False

    if len(ohlc):
        last_ts=ohlc.index[-1]
        equity_curve.append((last_ts, cash + qty*ohlc['Close'].iloc[-1]))
    ec = pd.DataFrame(equity_curve, columns=['Time','Equity']).set_index('Time')
    tr = pd.DataFrame(trades)
    return tr, ec


## Run for All Tickers

In [22]:

all_trades=[]; perf_rows=[]; all_ec={}
for t in TICKERS:
    print(f"Processing {t}...")
    df = fetch_1d(t, START_DATE, END_DATE, TZ)
    if df.empty:
        print(f"[WARN] {t}: no data");
        continue
    df.attrs['Ticker']=t
    df = add_indicators(df, FAST_DMA, SLOW_DMA, REGIME_DMA, RSI_LEN, ADX_LEN, ATR_LEN:=14)
    df = mark_crosses(df, FAST_DMA, SLOW_DMA)

    ent = entry_signal(
        df, FAST_DMA, SLOW_DMA,
        REQUIRE_CROSS_EXACTLY_T_MINUS_2, CROSS_WINDOW_AFTER,
        USE_FILTER_PRICE_ABOVE_200DMA, USE_FILTER_SLOW_SLOPE_POSITIVE,
        USE_FILTER_RSI, RSI_MIN,
        USE_FILTER_ADX, ADX_MIN,
        USE_FILTER_VOLUME_SURGE, VOL_LOOKBACK, VOL_FACTOR,
        USE_FILTER_ATR_BAND, ATR_MIN_PCT, ATR_MAX_PCT
    )
    exi = exit_signal(df, SLOW_DMA, PROTECTIVE_EXIT_ON_CLOSE_BELOW_20DMA)

    trades, ec = backtest_long_only(df, ent, exi, INITIAL_CAPITAL, RISK_PER_TRADE, SLIPPAGE_BPS, FEES_BPS)
    all_ec[t]=ec
    if not trades.empty: all_trades.append(trades.assign(Ticker=t))

    final_eq = ec['Equity'].iloc[-1]
    total_ret = final_eq/INITIAL_CAPITAL - 1.0
    dd = (ec['Equity']/ec['Equity'].cummax() - 1.0).min()
    perf_rows.append({'Ticker':t,'Trades':0 if trades.empty else len(trades)//2,
                      'FinalEquity':final_eq,'TotalReturn':total_ret,'MaxDrawdown':dd})

    # plot only if enabled
    if PLOT_LAST_N and PLOT_LAST_N>0:
        from datetime import datetime
        last_n = df.tail(PLOT_LAST_N).copy()
        # (plotting code omitted here to keep v2 lean; reuse from v1 if needed)

trades_df = pd.concat(all_trades, ignore_index=True) if all_trades else pd.DataFrame(
    columns=['Time','Ticker','Action','Qty','Price','Fees','PnL']
)
perf_df = pd.DataFrame(perf_rows).sort_values('TotalReturn', ascending=False)

trades_df.to_csv("dma_two_green_trades_v2.csv", index=False)
perf_df.to_csv("dma_two_green_performance_v2.csv", index=False)

print("\n=== Summary (v2) ===")
print(perf_df.to_string(index=False))
print("\nSaved: dma_two_green_trades_v2.csv, dma_two_green_performance_v2.csv")


Processing SBICARD.NS...


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_14784/292461430.py:89: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cross_cond = o['CrossUp'].shift(2).fillna(False)


Processing BDL.NS...


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_14784/292461430.py:89: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cross_cond = o['CrossUp'].shift(2).fillna(False)


Processing INDHOTEL.NS...


...

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_14784/292461430.py:89: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cross_cond = o['CrossUp'].shift(2).fillna(False)


Processing BEL.NS...


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_14784/292461430.py:89: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cross_cond = o['CrossUp'].shift(2).fillna(False)


Processing BAJAJFINSV.NS...

=== Summary (v2) ===
       Ticker  Trades  FinalEquity  TotalReturn  MaxDrawdown
 SOLARINDS.NS       1 86053.949534     0.721079    -0.056436
      MFSL.NS       1 70298.280462     0.405966    -0.041980
     DIXON.NS       1 57041.820934     0.140836    -0.087293
     NYKAA.NS       1 56383.988210     0.127680    -0.048577
  HDFCBANK.NS       3 53409.196291     0.068184    -0.054589
 ICICIBANK.NS       1 53394.693326     0.067894    -0.050688
MUTHOOTFIN.NS       1 52068.210176     0.041364    -0.029077
     PAYTM.NS       1 51534.376732     0.030688    -0.081564
   COFORGE.NS       2 51067.232205     0.021345    -0.181963
  SHREECEM.NS       0 50000.000000     0.000000     0.000000
       BDL.NS       0 50000.000000     0.000000     0.000000
       BEL.NS       0 50000.000000     0.000000     0.000000
   SBILIFE.NS       0 50000.000000     0.000000     0.000000
       SRF.NS       0 50000.000000     0.000000     0.000000
PERSISTENT.NS       0 50000.000000 

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_14784/292461430.py:89: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cross_cond = o['CrossUp'].shift(2).fillna(False)


## Suggested Defaults to Reduce Losing Trades
- Keep **USE_FILTER_PRICE_ABOVE_200DMA = True**
- Keep **USE_FILTER_SLOW_SLOPE_POSITIVE = True**
- Use **RSI_MIN = 50–55**
- Use **ADX_MIN = 18–25** depending on trendiness
- Enable **ATR band**; try `ATR_MIN_PCT=0.5` and `ATR_MAX_PCT=4.0`
- Consider enabling **volume surge** with `VOL_FACTOR=1.2–1.5`
- Keep **protective exit** on to avoid waiting for two red closes


# ➕ Groww Trading Add‑On (for v2)
This section integrates the v2 strategy with **Groww** for order planning and optional live execution.
- Keeps **all v2 parameters/filters** intact.
- Recomputes signals per ticker using the same v2 entry/exit rules.
- Builds a next‑day **order plan** from the last completed daily bar (IST).
- Optional **live** MARKET/CNC orders via Groww SDK.

> Run earlier v2 cells first (data functions). Then run these cells from top to bottom.

## Groww: Parameters & Auth

In [ ]:

# If first run on this machine, uncomment:
# !pip install --upgrade growwapi pytz python-dateutil

from datetime import datetime, timedelta
import pytz, uuid
from dateutil import parser as dateparser

# ---- Auth ----
AUTH_METHOD = "access_token"   # "access_token" or "key_secret"
ACCESS_TOKEN = ""              # paste if using access_token
API_KEY = ""                   # paste if using key_secret
API_SECRET = ""                # paste if using key_secret

# ---- Execution ----
DRY_RUN = True                 # True = paper; no real orders
LIVE_TRADING = False           # set True to actually place orders
ORDER_PRODUCT = "CNC"          # CNC for delivery
ORDER_TYPE = "MARKET"
VALIDITY = "DAY"

# ---- Universe mapping ----
# We will convert Yahoo symbols like 'RELIANCE.NS' to Groww trading_symbol 'RELIANCE'
def yahoo_to_groww(ts):
    return ts.replace(".NS","").replace(".BSE","")

from growwapi import GrowwAPI

def groww_login(auth_method, access_token=None, api_key=None, api_secret=None):
    if auth_method == "access_token":
        if not access_token:
            raise ValueError("Set ACCESS_TOKEN for AUTH_METHOD='access_token'")
        return GrowwAPI(access_token)
    elif auth_method == "key_secret":
        if not api_key or not api_secret:
            raise ValueError("Set API_KEY and API_SECRET for AUTH_METHOD='key_secret'")
        token = GrowwAPI.get_access_token(api_key=api_key, secret=api_secret)
        if not token:
            raise RuntimeError("Failed to fetch access token using key+secret")
        return GrowwAPI(token)
    else:
        raise ValueError("AUTH_METHOD must be 'access_token' or 'key_secret'")

groww = groww_login(AUTH_METHOD, ACCESS_TOKEN, API_KEY, API_SECRET)
print("Groww client ready.")
IST = pytz.timezone("Asia/Kolkata")
RUN_TAG = datetime.now(IST).strftime("%Y-%m-%d_%H-%M-%S")


## Market hours & holdings helpers

In [ ]:

def india_market_open_now(ts=None):
    now = ts or datetime.now(IST)
    if now.weekday() >= 5:  # Sat/Sun
        return False
    start = now.replace(hour=9, minute=15, second=0, microsecond=0)
    end   = now.replace(hour=15, minute=30, second=0, microsecond=0)
    return start <= now <= end

def get_holdings_map(client):
    try:
        h = client.get_holdings_for_user()
        mapping = {}
        items = []
        if isinstance(h, dict):
            items = h.get("holdings", h.get("payload", [])) or []
        elif isinstance(h, list):
            items = h
        for it in items:
            ts = it.get("trading_symbol") or it.get("symbol") or ""
            qty = int(it.get("quantity") or it.get("qty") or 0)
            if ts:
                mapping[ts] = mapping.get(ts, 0) + qty
        return mapping
    except Exception as e:
        print("[WARN] holdings fetch failed:", e)
        return {}

def place_market_order(client, trading_symbol, qty, exchange="NSE", segment="CASH",
                       product="CNC", validity="DAY", txn_type="BUY"):
    ref_id = str(uuid.uuid4())[:20]
    if DRY_RUN or not LIVE_TRADING:
        print(f"[DRY] {txn_type} {qty} {trading_symbol} {exchange}/{segment} {product} {validity} ref={ref_id}")
        return {"dry_run": True, "order_reference_id": ref_id}
    resp = client.place_order(
        trading_symbol=trading_symbol,
        quantity=int(qty),
        validity=getattr(client, f"VALIDITY_{validity}", "DAY"),
        exchange=getattr(client, f"EXCHANGE_{'NSE'}", "NSE"),
        segment=getattr(client, f"SEGMENT_{'CASH'}", "CASH"),
        product=getattr(client, f"PRODUCT_{product}", "CNC"),
        order_type=getattr(client, "ORDER_TYPE_MARKET"),
        transaction_type=getattr(client, f"TRANSACTION_TYPE_{'BUY' if txn_type=='BUY' else 'SELL'}"),
        order_reference_id=ref_id
    )
    print("[LIVE] place_order resp:", resp)
    return resp


## Build order plan from v2 signals (last completed day)

In [ ]:

import pandas as pd, numpy as np, os

SAVE_DIR = "groww_dma_orders"
os.makedirs(SAVE_DIR, exist_ok=True)

# We rely on v2 functions/params:
# - TICKERS
# - add_indicators, mark_crosses, entry_signal, exit_signal
# - START_DATE, END_DATE, TZ (v2 uses Asia/Kolkata already)

def compute_signals_for_symbol_yf(ticker):
    # Reuse v2's fetch_1d (yfinance) and pipeline to align with backtest signals
    df = fetch_1d(ticker, START_DATE, END_DATE, TZ)
    if df.empty:
        return df, None, None
    df = add_indicators(df, FAST_DMA, SLOW_DMA, REGIME_DMA, RSI_LEN, ADX_LEN, ATR_LEN:=14)
    df = mark_crosses(df, FAST_DMA, SLOW_DMA)
    ent = entry_signal(
        df, FAST_DMA, SLOW_DMA,
        REQUIRE_CROSS_EXACTLY_T_MINUS_2, CROSS_WINDOW_AFTER,
        USE_FILTER_PRICE_ABOVE_200DMA, USE_FILTER_SLOW_SLOPE_POSITIVE,
        USE_FILTER_RSI, RSI_MIN,
        USE_FILTER_ADX, ADX_MIN,
        USE_FILTER_VOLUME_SURGE, VOL_LOOKBACK, VOL_FACTOR,
        USE_FILTER_ATR_BAND, ATR_MIN_PCT, ATR_MAX_PCT
    )
    exi = exit_signal(df, SLOW_DMA, PROTECTIVE_EXIT_ON_CLOSE_BELOW_20DMA)
    return df, ent, exi

def last_completed_index(df):
    now = datetime.now(IST)
    if india_market_open_now(now):
        return df.index[-2] if len(df) >= 2 else None
    else:
        return df.index[-1] if len(df) >= 1 else None

holdings = get_holdings_map(groww)
orders_plan = []

for y_symbol in TICKERS:
    df, ent, exi = compute_signals_for_symbol_yf(y_symbol)
    if df.empty: 
        print(f"[WARN] no data for {y_symbol}")
        continue
    idx = last_completed_index(df)
    if idx is None:
        continue
    entry_sig = bool(ent.loc[idx]) if idx in ent.index else False
    exit_sig  = bool(exi.loc[idx]) if idx in exi.index else False
    close_px  = float(df.loc[idx, "Close"])
    groww_sym = yahoo_to_groww(y_symbol)
    held_qty  = holdings.get(groww_sym, 0)

    if entry_sig and held_qty <= 0:
        # Simple sizing: use fraction of capital like v2 backtest (RISK_PER_TRADE as fraction of equity)
        # v2 uses RISK_PER_TRADE = fraction of equity; here we translate to shares using close price
        alloc = INITIAL_CAPITAL * RISK_PER_TRADE
        qty = int(alloc // max(1e-6, close_px))
        if qty > 0:
            orders_plan.append({
                "action":"BUY","symbol":groww_sym,"qty":int(qty),
                "reason":"Entry signal (last completed day)","yahoo_symbol":y_symbol
            })
    if exit_sig and held_qty > 0:
        orders_plan.append({
            "action":"SELL","symbol":groww_sym,"qty":int(held_qty),
            "reason":"Exit signal (last completed day)","yahoo_symbol":y_symbol
        })

orders_df = pd.DataFrame(orders_plan)
if not orders_df.empty:
    path = os.path.join(SAVE_DIR, f"orders_plan_{RUN_TAG}.csv")
    orders_df.to_csv(path, index=False)
    print("Saved plan:", path)
orders_df.tail(20)


## Execute plan (MARKET/CNC)

In [ ]:

executed = []
if 'orders_df' not in globals() or orders_df.empty:
    print("No orders to execute.")
else:
    for _, r in orders_df.iterrows():
        resp = place_market_order(
            groww,
            trading_symbol=r['symbol'],
            qty=int(r['qty']),
            exchange="NSE", segment="CASH",
            product=ORDER_PRODUCT, validity=VALIDITY,
            txn_type=r['action']
        )
        executed.append({**r.to_dict(), "response": resp})
exec_df = pd.DataFrame(executed)
if not exec_df.empty:
    path = os.path.join(SAVE_DIR, f"executions_{RUN_TAG}.jsonl")
    with open(path, "w", encoding="utf-8") as f:
        for row in executed:
            f.write(json.dumps(row) + "\n")
    print("Saved executions:", path)
exec_df.tail(20)
